In [1]:
import numpy as np
import pandas as pd
import itertools
from tabulate import tabulate



##### Funkcja aktywacji

$
f(x) =
     \begin{cases}
       \text{0,} &\quad\text{for x}<\text{0} \\
       \text{1,} &\quad\text{for x}\ge0 \\
     \end{cases}
$



In [2]:
def activation_function(data:float)->float:
    return 0 if data<0 else 1

##### Obliczanie warstwy

$\begin{bmatrix}
1\\x_1\\x_2\\...\\x_m
\end{bmatrix}
\cdot$
$\begin{bmatrix}
w_{1.1} & w_{1.2} &\cdots & w_{1.n}
\\w_{2.1} & w_{2.2} &\cdots & w_{2.n}
\\w_{3.1} & w_{3.2} &\cdots & w_{3.n}
\\\vdots & \vdots & \ddots & \vdots
\\w_{m.1} & w_{m.2} &\cdots & w_{m.n}
\end{bmatrix}$
$=
\begin{bmatrix}
o_1\\o_2\\o_3\\...\\xn
\end{bmatrix}$

In [3]:
def compute_layer(inputs:list,weights:list,activation_function = None):
	inputs = np.array([1]  +inputs).transpose()
	weights = np.array(weights).transpose()
	output = inputs.dot(weights)
	if type(output) is not np.ndarray:
		output = [output]
	else:
		output.T
	if activation_function:
		output = [(activation_function(x),x) for x in output]
	return output

##### Wypisywanie danych warstwy

In [4]:
def print_layer(inputs, weights, bias,outputs):
	to_print = []
	for idx,input in enumerate(inputs):
		act_out,raw_out = outputs[idx]
		to_print.append([input,weights,bias,raw_out,act_out])
	print(tabulate(to_print,headers=["Wejście","Wagi","Bias","Bez aktywacji","Z aktywacją"]))



#### Bramka AND

In [5]:
bias = -1.5
weights = [1,1]
inputs = [list(p) for p in itertools.product([0,1],repeat=2)]
outputs = []
for idx,input in enumerate(inputs):
	out,raw_out = compute_layer(input,[bias]+weights,activation_function=activation_function)[0]
	outputs.append((out,raw_out))

print_layer(inputs,weights,bias,outputs)

Wejście    Wagi      Bias    Bez aktywacji    Z aktywacją
---------  ------  ------  ---------------  -------------
[0, 0]     [1, 1]    -1.5             -1.5              0
[0, 1]     [1, 1]    -1.5             -0.5              0
[1, 0]     [1, 1]    -1.5             -0.5              0
[1, 1]     [1, 1]    -1.5              0.5              1


#### Bramka OR

In [6]:
bias = -0.5
weights = [1,1]
inputs = [list(p) for p in itertools.product([0,1],repeat=2)]
outputs = []
for input in inputs:
	out,raw_out = compute_layer(input,[bias]+weights,activation_function=activation_function)[0]
	outputs.append((out,raw_out))

print_layer(inputs,weights,bias,outputs)

Wejście    Wagi      Bias    Bez aktywacji    Z aktywacją
---------  ------  ------  ---------------  -------------
[0, 0]     [1, 1]    -0.5             -0.5              0
[0, 1]     [1, 1]    -0.5              0.5              1
[1, 0]     [1, 1]    -0.5              0.5              1
[1, 1]     [1, 1]    -0.5              1.5              1


#### Bramka NOT

In [7]:
bias = 1
weights = [-2]
inputs = [list(p) for p in itertools.product([1,0],repeat=1)]
outputs = []
for input in inputs:
	out,raw_out = compute_layer(input,[bias]+weights,activation_function=activation_function)[0]
	outputs.append((out,raw_out))

print_layer(inputs,weights,bias,outputs)

Wejście    Wagi      Bias    Bez aktywacji    Z aktywacją
---------  ------  ------  ---------------  -------------
[1]        [-2]         1               -1              0
[0]        [-2]         1                1              1


#### Bramka XNOR
$
A	\odot B =(\overline{\rm A}\land \overline{\rm B}) \lor( A \land B)
$

$
\overline{\rm A}\land \overline{\rm B} \quad i \ \quad A \land B
$

In [8]:
bias = [[0.5],[-1.5]]
weights = [[-1,-1],[1,1]]
inputs = [list(p) for p in itertools.product([1,0],repeat=2)]
outputs = []
for input in inputs:
	results = compute_layer(input,np.concatenate((bias,weights),axis=1),activation_function=activation_function)
	outputs.append(([result[0]for result in results],[result[1]for result in results]))
print_layer(inputs,weights,bias,outputs)

Wejście    Wagi                Bias             Bez aktywacji    Z aktywacją
---------  ------------------  ---------------  ---------------  -------------
[1, 1]     [[-1, -1], [1, 1]]  [[0.5], [-1.5]]  [-1.5, 0.5]      [0, 1]
[1, 0]     [[-1, -1], [1, 1]]  [[0.5], [-1.5]]  [-0.5, -0.5]     [0, 0]
[0, 1]     [[-1, -1], [1, 1]]  [[0.5], [-1.5]]  [-0.5, -0.5]     [0, 0]
[0, 0]     [[-1, -1], [1, 1]]  [[0.5], [-1.5]]  [0.5, -1.5]      [1, 0]


$
(\overline{\rm A}\land \overline{\rm B}) \lor( A \land B)
$

In [9]:
bias = [-0.5]
weights = [1,1]
inputs = [x[0] for x in outputs]
outputs = []
for input in inputs:
	results = compute_layer(input,bias+weights,activation_function=activation_function)[0]
	outputs.append(results)

print_layer(inputs,weights,bias,outputs)



Wejście    Wagi    Bias      Bez aktywacji    Z aktywacją
---------  ------  ------  ---------------  -------------
[0, 1]     [1, 1]  [-0.5]              0.5              1
[0, 0]     [1, 1]  [-0.5]             -0.5              0
[0, 0]     [1, 1]  [-0.5]             -0.5              0
[1, 0]     [1, 1]  [-0.5]              0.5              1


#### Bramka XOR

$
A\oplus B = (A\lor B) \land (\overline{\rm A}\lor\overline{\rm B})
$

$
A\lor B \quad i \quad \overline{\rm A}\lor\overline{\rm B}
$

In [10]:
bias = [[-0.5],[1.5]]
weights = [[1,1],[-1,-1]]
inputs = [list(p) for p in itertools.product([1,0],repeat=2)]
outputs = []
for input in inputs:
	results = compute_layer(input,np.concatenate((bias,weights),axis=1),activation_function=activation_function)
	outputs.append(([result[0]for result in results],[result[1]for result in results]))
print_layer(inputs,weights,bias,outputs)


Wejście    Wagi                Bias             Bez aktywacji    Z aktywacją
---------  ------------------  ---------------  ---------------  -------------
[1, 1]     [[1, 1], [-1, -1]]  [[-0.5], [1.5]]  [1.5, -0.5]      [1, 0]
[1, 0]     [[1, 1], [-1, -1]]  [[-0.5], [1.5]]  [0.5, 0.5]       [1, 1]
[0, 1]     [[1, 1], [-1, -1]]  [[-0.5], [1.5]]  [0.5, 0.5]       [1, 1]
[0, 0]     [[1, 1], [-1, -1]]  [[-0.5], [1.5]]  [-0.5, 1.5]      [0, 1]


$(A\lor B) \land (\overline{\rm A}\lor\overline{\rm B})$


In [11]:
bias = [-1.5]
weights = [1,1]
inputs = [x[0] for x in outputs]
outputs = []
for input in inputs:
	results = compute_layer(input,bias+weights,activation_function=activation_function)[0]
	outputs.append(results)

print_layer(inputs,weights,bias,outputs)

Wejście    Wagi    Bias      Bez aktywacji    Z aktywacją
---------  ------  ------  ---------------  -------------
[1, 0]     [1, 1]  [-1.5]             -0.5              0
[1, 1]     [1, 1]  [-1.5]              0.5              1
[1, 1]     [1, 1]  [-1.5]              0.5              1
[0, 1]     [1, 1]  [-1.5]             -0.5              0
